In [16]:
import pandas as pd
import numpy as np
from datetime import datetime
import os   
import sys
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize
from scipy import interpolate
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import warnings
import statsmodels.api as sm
from scipy.optimize import fsolve
from scipy.stats import norm
from sklearn.decomposition import PCA
import warnings
from scipy.optimize import fsolve
import numpy.polynomial.polynomial as poly
warnings.filterwarnings(action='ignore')

In [17]:
tickers = pd.read_csv('top_50_tickers.csv')

In [18]:
tickers.ticker.unique()

array(['TSLA', 'NVDA', 'SMCI', 'PLTR', 'AMZN', 'META', 'BABA', 'LCID',
       'NIO', 'IBIT', 'AMD', 'GOOGL', 'MSFT', 'APP', 'MSTR', 'HOOD',
       'HIMS', 'F', 'GOOG', 'NKLA', 'AVGO', 'INTC', 'SOFI', 'MU', 'AMC',
       'WMT', 'SAVE', 'GM', 'SQ', 'TSM', 'COIN', 'MARA', 'ZI', 'PYPL',
       'ASTS', 'FSLR', 'IREN', 'BAC', 'CAVA', 'DKNG', 'NLY', 'XPEV',
       'NYCB', 'SNOW', 'FCX', 'TEM', 'SOUN', 'VRT', 'NFLX', 'RIVN'],
      dtype=object)

In [19]:
vols = pd.read_csv('./daily_variance_series_wrds.csv')
vols = vols[vols['SYM_ROOT'] == vols['symbol']]

In [20]:
vols.SYM_ROOT.nunique()

48

In [21]:
vols['month'] = pd.to_datetime(vols['DATE']).dt.month
vols['year'] = pd.to_datetime(vols['DATE']).dt.year
vols['ivol_q_log'] = np.log(vols['ivol_q'])

In [22]:
def moments(df):
    final_dfs = []
    for lag in range(1, 11):
        tmp = df.groupby(['SYM_ROOT', 'year', 'month'])['ivol_q_log']\
            .apply(lambda x: (((x - x.shift(lag))**2).sum() / (x.count() - lag)) ).to_frame(f'lag{lag}_mom')
        final_dfs.append(tmp)
    final_df = pd.concat(final_dfs, axis=1)
    return final_df

In [23]:
final = moments(vols)
final.dropna(inplace=True)

In [24]:
betas = pd.DataFrame(columns=['Betas'], index=final.index)
for idx,_ in final.iterrows():
    y = final.loc[idx].values
    X = np.log(range(1, 11)).reshape(-1,1)
    # X = sm.add_constant(X)
    mod = sm.OLS(y, X).fit()
    betas.loc[idx, 'Betas'] = mod.params
betas.reset_index(inplace=True)


In [37]:
betas[betas['Betas']>=0.5]

,SYM_ROOT,year,month,Betas
0,AMC,2014,1,0.6801783537114248
9,AMC,2014,10,0.5528995696897088
14,AMC,2015,3,0.8909532496157646
15,AMC,2015,4,0.9481944497048647
16,AMC,2015,5,0.8333481763645256
...,...,...,...,...
4652,WMT,2015,10,0.5497540198744904
4690,WMT,2018,12,0.6244070844211982
4705,WMT,2020,3,1.0741126779642438
4739,WMT,2023,1,0.6767466304778018


In [32]:
betas

,SYM_ROOT,year,month,Betas
0,AMC,2014,1,0.6801783537114248
1,AMC,2014,2,0.3044701508508652
2,AMC,2014,3,0.43894627795209695
3,AMC,2014,4,0.3677223030178472
4,AMC,2014,5,0.16202374674351883
...,...,...,...,...
4814,ZI,2023,3,4.1764630805925655
4815,ZI,2023,4,0.10833338739933156
4816,ZI,2023,5,0.1918529306195535
4817,ZI,2023,6,0.07078159863146065
